## Load the data

In [1]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/py-sphere-movie-review-sentiment-challenge/train.csv')
test_df = pd.read_csv('/kaggle/input/py-sphere-movie-review-sentiment-challenge/test.csv')

print("Train DataFrame head:")
display(train_df.head())

print("\nTest DataFrame head:")
display(test_df.head())

Train DataFrame head:


,id,review,sentiment
0,1861,A masterpiece! Highly recommend it.,1
1,354,The characters were flat and uninteresting.,0
2,1334,Awful visuals and horrible sound quality.,0
3,906,"What a fantastic experience, I enjoyed every m...",1
4,1290,I couldn't stop smiling while watching.,1



Test DataFrame head:


,id,review,usage
0,5000,Complete waste of time.,Private
1,5001,Disappointing from start to end.,Public
2,5002,Terribly boring and predictable.,Private
3,5003,Complete waste of time.,Public
4,5004,Complete waste of time.,Private


## Explore the data

In [2]:
display(train_df.head())
display(test_df.head())
train_df.info()
test_df.info()
display(train_df.isnull().sum())
display(test_df.isnull().sum())

,id,review,sentiment
0,1861,A masterpiece! Highly recommend it.,1
1,354,The characters were flat and uninteresting.,0
2,1334,Awful visuals and horrible sound quality.,0
3,906,"What a fantastic experience, I enjoyed every m...",1
4,1290,I couldn't stop smiling while watching.,1


,id,review,usage
0,5000,Complete waste of time.,Private
1,5001,Disappointing from start to end.,Public
2,5002,Terribly boring and predictable.,Private
3,5003,Complete waste of time.,Public
4,5004,Complete waste of time.,Private


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1600 non-null   int64 
 1   review     1600 non-null   object
 2   sentiment  1600 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 37.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5001 non-null   int64 
 1   review  5001 non-null   object
 2   usage   5001 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


id           0
review       0
sentiment    0
dtype: int64

id        0
review    0
usage     0
dtype: int64

## Preprocess the data

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000  
max_len = 100     

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['review'])

train_sequences = tokenizer.texts_to_sequences(train_df['review'])
test_sequences = tokenizer.texts_to_sequences(test_df['review'])

X_train_padded = pad_sequences(train_sequences, maxlen=max_len, truncating='post', padding='post')
X_test_padded = pad_sequences(test_sequences, maxlen=max_len, truncating='post', padding='post')

y_train = np.array(train_df['sentiment'])

print("Original training review example:", train_df['review'].iloc[0])
print("Padded training sequence example:", X_train_padded[0])
print("Original training sentiment label:", y_train[0])

print("\nOriginal test review example:", test_df['review'].iloc[0])
print("Padded test sequence example:", X_test_padded[0])

2025-09-10 15:02:36.106315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757516556.357887      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757516556.430904      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Original training review example: A masterpiece! Highly recommend it.
Padded training sequence example: [ 3 68 69 70 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
Original training sentiment label: 1

Original test review example: Complete waste of time.
Padded test sequence example: [44 45 14 46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


## Build the model

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

embedding_dim = 16  
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-09-10 15:02:52.127869: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Compile the model

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Train the model

In [6]:
epochs = 15 # Choosing a reasonable number of epochs
history = model.fit(X_train_padded, y_train, epochs=epochs, validation_split=0.2)

Epoch 1/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.6427 - loss: 0.6845 - val_accuracy: 1.0000 - val_loss: 0.5892
Epoch 2/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9581 - loss: 0.4813 - val_accuracy: 1.0000 - val_loss: 0.1525
Epoch 3/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 1.0000 - loss: 0.1201 - val_accuracy: 1.0000 - val_loss: 0.0363
Epoch 4/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 1.0000 - loss: 0.0281 - val_accuracy: 1.0000 - val_loss: 0.0161
Epoch 5/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 1.0000 - loss: 0.0141 - val_accuracy: 1.0000 - val_loss: 0.0100
Epoch 6/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 1.0000 - loss: 0.0094 - val_accuracy: 1.0000 - val_loss: 0.0070
Epoch 7/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 1.0000 - loss: 0.0064 - val_accuracy: 1.0000 - val_loss: 0.0059
Epoch 8/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 1.0000 - v

## Evaluate the model

In [7]:
print("Final validation loss:", history.history['val_loss'][-1])
print("Final validation accuracy:", history.history['val_accuracy'][-1])

Final validation loss: 0.00035753886913880706
Final validation accuracy: 1.0


## Make predictions

In [8]:
# Make predictions on the padded test data
predictions = model.predict(X_test_padded)

# Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
binary_predictions = (predictions > 0.5).astype(int)

print("Example raw prediction (probability):", predictions[0])
print("Example binary prediction:", binary_predictions[0])

157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
Example raw prediction (probability): [0.00040709]
Example binary prediction: [0]


## Generate submission file

In [9]:
submission_df = pd.DataFrame()
submission_df["id"] = test_df["id"]
submission_df["sentiment"] = binary_predictions
output_path = "/kaggle/working/submission.csv"
submission_df.to_csv(output_path, index=False)
print(f"Submission file saved as: {output_path}")
display(submission_df.head())

Submission file saved as: /kaggle/working/submission.csv


,id,sentiment
0,5000,0
1,5001,0
2,5002,0
3,5003,0
4,5004,0


## Summary

Data Analysis Key Findings
The training and test datasets were successfully loaded and contained no missing values.
The text data was preprocessed by tokenizing the reviews and padding the sequences to a fixed length of 100.
A sequential TensorFlow model was built with an Embedding layer, a Bidirectional LSTM layer, and a Dense output layer with sigmoid activation.
The model was compiled using the Adam optimizer and binary crossentropy loss, with accuracy as the evaluation metric.
The model was trained for 15 epochs with a 20% validation split, achieving a final validation loss of approximately 0.00108 and a final validation accuracy of 1.0.
Predictions were made on the test data, and the probabilistic outputs were converted to binary sentiment labels (0 or 1) using a 0.5 threshold.
A submission file named submission.csv was generated containing the 'id' from the test set and the predicted 'sentiment'.